## Import

In [ ]:
import random
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from numba import cuda
import os

In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())    # cuda 사용 가능 여부 확인
print(torch.cuda.device_count())    # 사용 가능한 GPU 개수 확인
device = cuda.get_current_device()  # 현재 사용하고 있는 GPU 확인
#device.reset()                     # GPU 캐시 리셋
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'    # GPU out of memory error 발생 시 주석 처리
os.environ['CUDA_VISIBLE_DEVICES'] = '0'    # 사용할 GPU 번호 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')     # cuda 사용 가능 여부에 따라 device 정보 저장

## Hyperparameter Setting

In [ ]:
CFG = {
    'TRAIN_WINDOW_SIZE':90, # 90일치로 학습
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':3,    # 학습횟수
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':1024,
    'SEED':41   # 시드 고정
}

In [ ]:
SKIP_COUNT = 1

In [ ]:
def seed_everything(seed):  # Seed 고정 함수
    random.seed(seed)      # random seed 고정
    os.environ['PYTHONHASHSEED'] = str(seed)    # PYTHONHASHSEED 값 설정
    np.random.seed(seed)    # numpy seed 고정
    torch.manual_seed(seed) # torch seed 고정
    torch.cuda.manual_seed(seed)    # torch cuda seed 고정
    torch.backends.cudnn.deterministic = True   # torch cudnn seed 고정
    torch.backends.cudnn.benchmark = True   # cudnn을 빠르게 하기 위한 옵션으로, 연산 진행시 어떤 알고리즘을 쓸지를 정하는 부분이다.

seed_everything(CFG['SEED']) # Seed 고정 함수실행

### 데이터 불러오기

In [ ]:
train_data = pd.read_csv('./data/brand_keyword_cnt.csv')
train_data = train_data[::SKIP_COUNT]
train_data.head(5)

### 데이터 전처리

In [ ]:
# 2번째방법 : min-max scaling
# 숫자형 변수들의 min-max scaling을 수행하는 코드입니다.
numeric_cols = train_data.columns[1:]
# 칵 column의 min 및 max 계산
min_values = train_data[numeric_cols].min(axis=1)
max_values = train_data[numeric_cols].max(axis=1)
# 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 1로 대체
ranges = max_values - min_values
ranges[ranges == 0] = 1
# min-max scaling 수행
train_data[numeric_cols] = (train_data[numeric_cols].subtract(min_values, axis=0)).div(ranges, axis=0)
# max와 min 값을 dictionary 형태로 저장
scale_min_dict = min_values.to_dict()
scale_max_dict = max_values.to_dict()

In [ ]:
train_data.fillna(0, inplace=True)

In [ ]:
# Label Encoding 문자형 변수를 숫자로 인코딩
label_encoder = LabelEncoder()  # sklearn.preprocessing 패키지의 LabelEncoder() 함수를 사용하여 라벨 인코더 생성    
label_encoder.fit((train_data['브랜드']))
train_data['브랜드'] = label_encoder.transform(train_data['브랜드'])

In [ ]:
def make_train_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
    '''
    학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성
    data : 일별 판매량
    train_size : 학습에 활용할 기간
    predict_size : 추론할 기간
    '''
    num_rows = len(data)    # 제품의 개수
    window_size = train_size + predict_size   # 학습 기간 + 예측 기간으로 윈도우사이즈 결정
    
    #빈 배열 생성
    input_data = np.empty((num_rows * (len(data.columns) - window_size + 1), train_size, len(data.iloc[0, :1]) + 1))   
    target_data = np.empty((num_rows * (len(data.columns) - window_size + 1), predict_size))   
    
    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :1])    
        sales_data = np.array(data.iloc[i, 1:]) 
        
        for j in range(len(sales_data) - window_size + 1):
            window = sales_data[j : j + window_size]    # 현재 조합에 해당하는 윈도우 추출
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))   # encode_info를 train_size만큼 반복하여 학습 데이터와 결합한 뒤, 윈도우의 앞쪽 train_size 개의 데이터와 함께 temp_data로 저장
            input_data[i * (len(data.columns) - window_size + 1) + j] = temp_data   # input_data에 temp_data 저장
            target_data[i * (len(data.columns) - window_size + 1) + j] = window[train_size:]    # target_data에 윈도우의 뒤쪽 predict_size 개의 데이터 저장
    
    return input_data, target_data

In [ ]:
def make_predict_data(data, train_size=CFG['TRAIN_WINDOW_SIZE']):
    '''
    평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성
    data : 일별 판매량
    train_size : 추론을 위해 필요한 일별 판매량 기간 (= 학습에 활용할 기간)
    '''
    num_rows = len(data)    # 제품의 개수 ( =행의 개수)
    
    input_data = np.empty((num_rows, train_size, len(data.iloc[0, :1]) + 1))    # 빈 배열 생성
    
    for i in tqdm(range(num_rows)):   
        encode_info = np.array(data.iloc[i, :1])    
        sales_data = np.array(data.iloc[i, -train_size:])   # 실제 판매량 정보
        
        window = sales_data[-train_size : ]   # 추론을 위한 일별 판매량 기간만큼의 데이터를 window로 저장
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))   # encode_info를 train_size만큼 반복하고, 추론 기간에 해당하는 데이터와 함께 열 방향으로 합쳐서 temp_data를 생성
        input_data[i] = temp_data
    
    return input_data

In [12]:
train_input, train_target = make_train_data(train_data)
test_input = make_predict_data(train_data)

  0%|          | 0/3170 [00:00<?, ?it/s]

In [13]:
# Train / Validation Split (고정)
# 맨 뒤 데이터의 20%를 Validation Set으로 사용
data_len = len(train_input)
val_input = train_input[-int(data_len*0.2):]
val_target = train_target[-int(data_len*0.2):]
train_input = train_input[:-int(data_len*0.2)]
train_target = train_target[:-int(data_len*0.2)]

In [14]:
train_input.shape, train_target.shape, val_input.shape, val_target.shape, test_input.shape

((887600, 90, 2), (887600, 21), (221900, 90, 2), (221900, 21), (3170, 90, 2))

### Custom Dataset

In [15]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])
    
    def __len__(self):
        return len(self.X)

In [16]:
'''
Dataset 은 데이터셋의 특징(feature)을 가져오고 하나의 샘플에 정답(label)을 지정하는 일을 한 번에 합니다.
모델을 학습할 때, 일반적으로 샘플들을 《미니배치(minibatch)》로 전달하고, 매 에폭(epoch)마다 데이터를 다시 섞어서 과적합(overfit)을 막고,
Python의 multiprocessing 을 사용하여 데이터 검색 속도를 높이려고 합니다.
DataLoader 는 간단한 API로 이러한 복잡한 과정들을 추상화한 순회 가능한 객체(iterable)입니다.
'''

train_dataset = CustomDataset(train_input, train_target)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_input, val_target)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

### 모델 선언

In [17]:
# Single hidden layer LSTM
class BaseModel(nn.Module):
    def __init__(self, input_size=2, hidden_size=512, output_size=CFG['PREDICT_SIZE']):
        super(BaseModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)  
        self.fc = nn.Sequential(                        # fully connected layer
            nn.Linear(hidden_size, hidden_size//2),     # 완전 연결 레이어 1
            nn.ReLU(),                                  # 활성화 함수
            nn.Dropout(),                               # 드롭아웃
            nn.Linear(hidden_size//2, output_size),     # 완전 연결 레이어 2
        )
        self.actv = nn.ReLU()
    
    def forward(self, x):
        # forward pass
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size, x.device) # hidden state 초기화
        
        # LSTM layer
        lstm_out, hidden = self.lstm(x, hidden) #LSTM 시행
         
        # Only use the last output sequence
        last_output = lstm_out[:, -1, :]    # LSTM 레이어의 출력 중 마지막 값만 사용
        
        # Fully connected layer
        output = self.actv(self.fc(last_output))    # 마지막 타임스텝의 출력값을 fully connected layer에 통과
        
        return output.squeeze(1)    # 모델의 출력을 1차원으로 변환
    
    def init_hidden(self, batch_size, device):
        # Initialize hidden state and cell state
        return (torch.zeros(1, batch_size, self.hidden_size, device=device),    # hidden state 초기화
                torch.zeros(1, batch_size, self.hidden_size, device=device))    # cell state 초기화


### 모델 학습

In [18]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    
    with torch.no_grad():   # 검증 단계이므로 역전파가 필요없음, no_grad()를 통해 메모리 사용량을 줄임
        for X, Y in tqdm(iter(val_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            output = model(X)
            loss = criterion(output, Y)
            
            val_loss.append(loss.item())
    return np.mean(val_loss)

In [19]:
def train(model, optimizer, train_loader, val_loader, device):
    '''
    model : 학습할 모델
    optimizer : 최적화 알고리즘
    train_loader : 학습 데이터로더
    val_loader : 검증 데이터로더
    device : 모델 및 데이터 연산에 사용할 디바이스 (GPU 또는 CPU)
    '''
    model.to(device)
    criterion = nn.MSELoss().to(device) # Mean Squared Error
    best_loss = 9999999
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []     # 학습 데이터셋의 손실값 저장
        train_mae = []      # 학습 데이터의 MAE(Mean Absolute Error) 저장
        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            optimizer.zero_grad()   # 이전 iteration에서 계산된 gradient를 초기화
            
            output = model(X)
            loss = criterion(output, Y)
            
            loss.backward()        # 역전파로 gradient 계산
            optimizer.step()    # 계산된 gradient로 모델의 parameter를 업데이트
                        
            train_loss.append(loss.item())
        
        val_loss = validation(model, val_loader, criterion, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')
        
        if best_loss > val_loss:    # 검증 데이터의 손실값 비교후 가장 적은 모델을 저장
            best_loss = val_loss
            best_model = model
            print('Model Saved')
    return best_model

## Run !!

In [ ]:
model = BaseModel()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])    # Adam optimizer를 사용
infer_model = train(model, optimizer, train_loader, val_loader, device)   # 학습 진행

In [26]:
def saveModel(model, PATH):
    torch.save(model.state_dict(), PATH)


saveModel(infer_model, './model/BrandKeyword_LSTM.pt')

In [25]:
def loadModel(model_path):
    loaded_model = BaseModel()
    loaded_model.load_state_dict(torch.load(model_path))
    return loaded_model

infer_model = loadModel('./model/BrandKeyword_LSTM.pt')
model = BaseModel()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])    # Adam optimizer를 사용

  0%|          | 0/867 [00:00<?, ?it/s]

  0%|          | 0/217 [00:00<?, ?it/s]

Epoch : [1] Train Loss : [0.02040] Val Loss : [0.01775]
Model Saved


  0%|          | 0/867 [00:00<?, ?it/s]

  0%|          | 0/217 [00:00<?, ?it/s]

Epoch : [2] Train Loss : [0.01752] Val Loss : [0.01842]


  0%|          | 0/867 [00:00<?, ?it/s]

  0%|          | 0/217 [00:00<?, ?it/s]

Epoch : [3] Train Loss : [0.01735] Val Loss : [0.01874]


## 모델 추론

In [27]:
test_dataset = CustomDataset(test_input, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [28]:
def inference(model, test_loader, device):
    predictions = []
    
    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            X = X.to(device)
            output = model(X)
            output = output.cpu().numpy()       # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환
            predictions.extend(output)
    
    return np.array(predictions)

In [29]:
pred = inference(infer_model, test_loader, device)

  0%|          | 0/4 [00:00<?, ?it/s]

In [30]:
# 추론 결과를 inverse scaling
for idx in range(len(pred)):
    pred[idx, :] = pred[idx, :] * (scale_max_dict[idx*SKIP_COUNT] - scale_min_dict[idx*SKIP_COUNT]) + scale_min_dict[idx*SKIP_COUNT]


In [31]:
ids = np.arange(len(pred)).reshape(-1, 1)
pred_ids = np.hstack((ids, pred))

In [32]:
print(pred.shape)
print(pred_ids.shape)
print(pred_ids)

(3170, 21)
(3170, 22)
[[0.00000000e+00 3.82713675e-01 3.93186212e-01 ... 4.02351737e-01
  4.04668510e-01 4.08492208e-01]
 [1.00000000e+00 1.06412506e+01 1.07437668e+01 ... 1.05361385e+01
  1.06976166e+01 1.06593409e+01]
 [2.00000000e+00 9.57895398e-01 3.88316453e-01 ... 1.03484964e+00
  1.17892313e+00 7.97882676e-01]
 ...
 [3.16700000e+03 4.10876013e-02 4.01737280e-02 ... 3.92210968e-02
  4.06862944e-02 4.10946235e-02]
 [3.16800000e+03 1.63184240e-01 1.64105028e-01 ... 1.60452664e-01
  1.63450390e-01 1.64882734e-01]
 [3.16900000e+03 5.06647491e+00 5.06373978e+00 ... 4.97793913e+00
  4.99976778e+00 5.02520752e+00]]


In [44]:
inversed_brand = label_encoder.inverse_transform(pred_ids[:, 0].astype(int))
inversed_brand = inversed_brand.reshape(-1, 1)
pred_brand = np.hstack([inversed_brand, pred])

## Submission

In [46]:
pred_brand_df = pd.DataFrame(pred_brand, columns=['브랜드'] + pd.date_range(start='2023-04-05', end='2023-04-25').strftime('%Y-%m-%d').tolist())

In [47]:
pred_brand_df.to_csv('./data/BrandKeywordOutput.csv', index=False)